In [1]:
#get the goods

import pandas as pd
import geopandas as gpd
import fiona
import random
import numpy as np
import rasterio
from shapely.geometry import Point

In [2]:
#creating a function that will calculate the extent of a polygon automatically

def get_extent(poly):
    extent = poly.bounds
    dict = {'minX' : extent[0],
            'minY' : extent[1],
            'maxX' : extent[2],
            'maxY' : extent[3]
           }
    return(dict)

In [3]:
#preparing data

data_dir = '../lab3/lab3.gpkg'
master_crs = "EPSG:26913"
density = 0.05
random.seed(0)
index = 0

layers = fiona.listlayers(data_dir)
watersheds = [] 
for layer_name in layers:
    if layer_name.startswith('wdb'):
        watersheds.append(layer_name)

In [4]:
#parts 1 and 2 done concurrently

for each_layer in watersheds:
    sample_points =  {'huc_id':[],
                      'geometry':[]
                     }
    w = gpd.read_file(data_dir, layer = each_layer)
    for idx, row in w.iterrows():
        extent = get_extent(row.geometry)
        area_km = row['geometry'].area/10**6
        n = round(density * area_km)
        for i in range(n):
            intersects = False
            while intersects == False:
                x = random.uniform(extent['minX'], extent['maxX'])
                y = random.uniform(extent['minY'], extent['maxY'])
                point = Point(x,y)
                results = row['geometry'].intersects(point)
                if results == True:
                    sample_points['huc_id'].append(row[0][:8])
                    sample_points['geometry'].append(point)
                    intersects = True
    points_gdf = gpd.GeoDataFrame(sample_points, crs = master_crs, geometry = 'geometry')
    soils_gdf = gpd.read_file(data_dir, crs = master_crs, layer = 'ssurgo_mapunits_lab3')
    intersect = gpd.overlay(points_gdf, soils_gdf, how = 'intersection')
    results = intersect.groupby(['huc_id'])['aws0150'].mean()
    if index < 1:
        index += 1
        print("St. Vrain watershed - HUC8 derived point sample: mean aws0150 =", results[0],
              "\nBig Thompson watershed - HUC8 derived point sample: mean aws0150 =", results[1],
              "\nCache La Poudre watershed - HUC8 derived point sample: mean aws0150 =", results[2])
    else:
        print("St. Vrain watershed - HUC12 derived point sample mean aws0150 =", results[0],
              "\nBig Thompson watershed - HUC12 derived point sample: mean aws0150 =", results[1],
              "\nCache La Poudre watershed - HUC12 derived point sample: mean aws0150 =", results[2])

St. Vrain watershed - HUC8 derived point sample: mean aws0150 = 11.473189655172416 
Big Thompson watershed - HUC8 derived point sample: mean aws0150 = 7.8777884615384615 
Cache La Poudre watershed - HUC8 derived point sample: mean aws0150 = 11.386864406779662
St. Vrain watershed - HUC12 derived point sample mean aws0150 = 10.84136 
Big Thompson watershed - HUC12 derived point sample: mean aws0150 = 10.492456140350878 
Cache La Poudre watershed - HUC12 derived point sample: mean aws0150 = 10.6236820083682
